In [6]:
import pandas as pd
from fuzzywuzzy import fuzz

# SIC Codes

In [8]:
ibes1 = pd.read_parquet('../data/raw/sic_processing_files/ibes_data_for_sic.parquet')

In [9]:
ibes1.shape

(93507, 4)

In [10]:
# summarizse data by last and first date in the mentioned in the table
ibes1_date = ibes1.groupby(["ticker", "cusip"]).sdates.agg(["min", "max"]).reset_index().rename(columns={"min": "fdate", "max": "ldate"})

# add data information in the main table
ibes2 = pd.merge(ibes1,ibes1_date, how = "left", on = ["ticker", "cusip"])
ibes2 = ibes2.sort_values(by = ["ticker", "cusip", "sdates"])

# drop column that we dont need anymore and filter data based picking only the latest date erwähnt
ibes2 = ibes2.loc[ibes2.sdates == ibes2.ldate].drop(["sdates"], axis=1)
ibes2.head()


,ticker,cusip,cname,fdate,ldate
1,0000,87482X10,TALMER BANCORP,2014-02-20,2014-03-20
6,0001,26878510,EP ENGR CORP,2014-02-20,2019-10-17
7,0004,02504D10,AMERICAN CAPITAL,2014-02-20,2014-02-20
8,000R,14163310,CARECOM,2014-02-20,2014-02-20
9,000V,15117E10,CELLADON,2014-03-20,2014-03-20


In [11]:
ibes2.shape

(39236, 5)

In [12]:
crsp1 = pd.read_parquet('../data/raw/sic_processing_files/crsp_link.parquet')

In [13]:
# first namedt
crsp1_fnamedt = crsp1.groupby(['permno','ncusip']).namedt.min().reset_index()

# last nameenddt
crsp1_lnameenddt = crsp1.groupby(['permno','ncusip']).nameenddt.max().reset_index()

# merge both 
crsp1_dtrange = pd.merge(crsp1_fnamedt, crsp1_lnameenddt, \
                          on = ['permno','ncusip'], how='inner')

# replace namedt and nameenddt with the version from the dtrange
crsp1 = crsp1.drop(['namedt'],axis=1).rename(columns={'nameenddt':'enddt'})
crsp2 = pd.merge(crsp1, crsp1_dtrange, on =['permno','ncusip'], how='inner')

# keep only most recent company name
crsp2 = crsp2.loc[crsp2.enddt == crsp2.nameenddt].drop(['enddt'], axis=1)

In [14]:
# 1.3 Create CUSIP Link Table

# Link by full cusip, company names and dates
link1_1 = pd.merge(ibes2, crsp2, how='inner', left_on='cusip', right_on='ncusip').sort_values(['ticker','permno','ldate'])


In [15]:
# Keep link with most recent company name
link1_1_tmp = link1_1.groupby(['ticker','permno']).ldate.max().reset_index()
link1_2 = pd.merge(link1_1, link1_1_tmp, how='inner', on =['ticker', 'permno', 'ldate'])
link1_2['name_ratio'] = link1_2.apply(lambda x: fuzz.token_set_ratio(x.comnam, x.cname), axis=1)

# 10% percentile of the company name distance
name_ratio_p10 = link1_2.name_ratio.quantile(0.10)
name_ratio_p10

67.0

In [16]:
# Function to assign score for companies matched by:
# full cusip and passing name_ratio
# or meeting date range requirement

def score1(row):
    if (row['fdate']<=row['nameenddt']) & (row['ldate']>=row['namedt']) & (row['name_ratio'] >= name_ratio_p10):
        score = 0
    elif (row['fdate']<=row['nameenddt']) & (row['ldate']>=row['namedt']):
        score = 1
    elif row['name_ratio'] >= name_ratio_p10:
        score = 2
    else:
        score = 3
    return score

# assign size portfolio
link1_2['score']=link1_2.apply(score1, axis=1)
link1_2 = link1_2[['ticker','permno','cname','comnam','name_ratio','score']]
link1_2 = link1_2.drop_duplicates()

In [17]:
link1_2.head()

,ticker,permno,cname,comnam,name_ratio,score
0,0000,14471,TALMER BANCORP,TALMER BANCORP INC,100,0
1,0001,14392,EP ENGR CORP,E P ENERGY CORP,67,0
2,0004,14418,AMERICAN CAPITAL,AMERICAN CAPITAL SR FLOATING LTD,100,0
3,000R,14378,CARECOM,CARE COM INC,74,0
4,000V,14423,EIGER,EIGER BIOPHARMACEUTICALS INC,100,0


In [18]:
link1_2.groupby(['score']).score.count()

score
0    18880
1     1800
2      679
3       81
Name: score, dtype: int64

In [20]:
##########################
# Step 2: Link by TICKER #
##########################

# Find links for the remaining unmatched cases using Exchange Ticker 

# Identify remaining unmatched cases 
nomatch1 = pd.merge(ibes2[['ticker']], link1_2[['permno','ticker']], on='ticker', how='left')
nomatch1 = nomatch1.loc[nomatch1.permno.isnull()].drop(['permno'], axis=1).drop_duplicates()

# Add IBES identifying information

ibesid = pd.read_parquet('../data/raw/sic_processing_files/ibes_id_data.parquet')
ibesid = ibesid.loc[ibesid.oftic.notna()]

nomatch2 = pd.merge(nomatch1, ibesid, how='inner', on=['ticker'])
nomatch2.shape

(16919, 5)

In [21]:
# Create first and last 'start dates' for Exchange Tickers
# Label date range variables and keep only most recent company name
nomatch3 = nomatch2.groupby(['ticker', 'oftic']).sdates.agg(['min', 'max']).reset_index().rename(columns={'min':'fdate', 'max':'ldate'})

nomatch3 = pd.merge(nomatch2, nomatch3, how='left', on=['ticker','oftic'])

nomatch3 = nomatch3.loc[nomatch3.sdates == nomatch3.ldate]
nomatch3.shape

(8542, 7)

In [23]:
# Get entire list of CRSP stocks with Exchange Ticker information
crsp_n1 = pd.read_parquet('../data/raw/sic_processing_files/crsp_w_ticker_info.parquet')

crsp_n1 = crsp_n1.loc[crsp_n1.ticker.notna()].sort_values(by=['permno','ticker','namedt'])

In [24]:
# Arrange effective dates for link by Exchange Ticker

crsp_n1_namedt = crsp_n1.groupby(['permno','ticker']).namedt.min().reset_index().rename(columns={'min':'namedt'})
crsp_n1_nameenddt = crsp_n1.groupby(['permno','ticker']).nameenddt.max().reset_index().rename(columns={'max':'nameenddt'})

crsp_n1_dt = pd.merge(crsp_n1_namedt, crsp_n1_nameenddt, how = 'inner', on=['permno','ticker'])

crsp_n1 = crsp_n1.rename(columns={'namedt': 'namedt_ind', 'nameenddt':'nameenddt_ind'})

crsp_n2 = pd.merge(crsp_n1, crsp_n1_dt, how ='left', on = ['permno','ticker'])

crsp_n2 = crsp_n2.rename(columns={'ticker':'crsp_ticker'})
crsp_n2 = crsp_n2.loc[crsp_n2.nameenddt_ind == crsp_n2.nameenddt].drop(['namedt_ind', 'nameenddt_ind'], axis=1)

# Merge remaining unmatched cases using Exchange Ticker 
# Note: Use ticker date ranges as exchange tickers are reused overtime

link2_1 = pd.merge(nomatch3, crsp_n2, how='inner', left_on=['oftic'], right_on=['crsp_ticker'])
link2_1 = link2_1.loc[(link2_1.ldate>=link2_1.namedt) & (link2_1.fdate<=link2_1.nameenddt)]


In [25]:
# Score using company name using 6-digit CUSIP and company name spelling distance
link2_1['name_ratio'] = link2_1.apply(lambda x: fuzz.token_set_ratio(x.comnam, x.cname), axis=1)

link2_2 = link2_1
link2_2['cusip6'] = link2_2.apply(lambda x: x.cusip[:6], axis=1)
link2_2['ncusip6'] = link2_2.apply(lambda x: x.ncusip[:6], axis=1)


In [26]:
# Score using company name using 6-digit CUSIP and company name spelling distance
def score2(row):
    if (row['cusip6']==row['ncusip6']) & (row['name_ratio'] >= name_ratio_p10):
        score = 0
    elif (row['cusip6']==row['ncusip6']):
        score = 4
    elif row['name_ratio'] >= name_ratio_p10:
        score = 5
    else:
        score = 6
    return score

# assign size portfolio
link2_2['score']=link2_2.apply(score2, axis=1)

In [27]:
# Some companies may have more than one TICKER-PERMNO link
# so re-sort and keep the case (PERMNO & Company name from CRSP)
# that gives the lowest score for each IBES TICKER 

link2_2 = link2_2[['ticker','permno','cname','comnam', 'name_ratio', 'score']].sort_values(by=['ticker','score'])
link2_2_score = link2_2.groupby(['ticker']).score.min().reset_index()

link2_3 = pd.merge(link2_2, link2_2_score, how='inner', on=['ticker', 'score'])
link2_3 = link2_3[['ticker','permno','cname','comnam','score']].drop_duplicates()

In [28]:
ibes_crsp_link = pd.concat([link1_2, link2_3])
ibes_crsp_link.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21808 entries, 0 to 374
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ticker      21808 non-null  object 
 1   permno      21808 non-null  int64  
 2   cname       21808 non-null  object 
 3   comnam      21808 non-null  object 
 4   name_ratio  21440 non-null  float64
 5   score       21808 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.2+ MB


In [30]:
crisp_compustat_link = pd.read_parquet('../data/raw/sic_processing_files/crisp-computsat-link.parquet')

In [31]:
crisp_compustat_link.shape

(31952, 12)

In [32]:
crisp_compustat_link.head()

,gvkey,conm,tic,cusip,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,A & E PLASTIK PAK INC,AE.2,000032102,3089,P,01,LU,25881,23369,1970-11-13,1978-06-30
1,1001,A & M FOOD SERVICES INC,AMFD.,000165100,5812,P,01,LU,10015,6398,1983-09-20,1986-07-31
2,1002,AAI CORP,AAIC.1,000352104,3825,C,01,LC,10023,22159,1972-12-14,1973-06-05
3,1003,A.A. IMPORTING CO INC,ANTQ,000354100,5712,C,01,LU,10031,6672,1983-12-07,1989-08-16
4,1004,AAR CORP,AIR,000361105,5080,P,01,LU,54594,20000,1972-04-24,E


In [33]:
# for how many permno from ibes_crsp_link do we have a link in crisp_compustat_link lpermno  and if not matched in LPERMCO
ibes_crsp_link['permno'].isin(crisp_compustat_link['LPERMNO']).value_counts()

permno
True     21130
False      678
Name: count, dtype: int64

In [34]:
# create linking table 
linking_table = pd.merge(ibes_crsp_link, crisp_compustat_link, how='left', left_on='permno', right_on='LPERMNO')
linking_table.head()

,ticker,permno,cname,comnam,name_ratio,score,gvkey,conm,tic,cusip,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,0000,14471,TALMER BANCORP,TALMER BANCORP INC,100.0,0,192458.0,TALMER BANCORP INC,TLMR,87482X101,6020.0,P,01,LC,14471.0,54747.0,2014-02-12,2016-09-30
1,0001,14392,EP ENGR CORP,E P ENERGY CORP,67.0,0,18660.0,EP ENERGY CORP,EPEGQ,268785102,1311.0,P,01,LC,14392.0,54697.0,2014-01-17,2019-04-30
2,0004,14418,AMERICAN CAPITAL,AMERICAN CAPITAL SR FLOATING LTD,100.0,0,19545.0,AMERICAN CAPITAL SR FLTG LTD,ACSF,02504D108,6797.0,P,01,LC,14418.0,54710.0,2014-01-16,2018-09-28
3,000R,14378,CARECOM,CARE COM INC,74.0,0,19352.0,CARE.COM INC,CRCM,141633107,7370.0,P,01,LC,14378.0,54694.0,2014-01-24,2020-02-28
4,000V,14423,EIGER,EIGER BIOPHARMACEUTICALS INC,100.0,0,18929.0,CELLADON CORP,CLDN,15117E107,2836.0,P,01,LC,14423.0,54714.0,2014-01-30,2016-03-22


In [35]:
linking_table = linking_table[["ticker", "sic"]]

In [36]:
linking_table.loc[linking_table["sic"].isna(), "sic"] = -1
linking_table["sic"] = linking_table["sic"].astype(str).str[:2]
linking_table["sic"] = linking_table["sic"].astype(int)

In [37]:
linking_table = linking_table.drop_duplicates()

In [38]:
linking_table['total_ticker_count'] = linking_table.groupby('ticker')['ticker'].transform('count')

In [39]:
grouped = linking_table.groupby('ticker').agg({'sic': 'max', 'total_ticker_count': 'first'}).reset_index()

# The DataFrame is already in the format we need based on your requirements
new_df_adjusted = grouped[['ticker', 'sic']]
new_df_adjusted

,ticker,sic
0,0000,60
1,0001,13
2,0004,67
3,000R,73
4,000V,28
...,...,...
21098,ZXIS,35
21099,ZXZX,37
21100,ZY,56
21101,ZYNE,28


In [41]:
new_df_adjusted.to_parquet('../data/processed/sic_linking_table.parquet')

# Experience

### step 1 - preproceed original data

In [42]:
def convert_to_datetime(df):
    """
    Service function for changing columns to datetime format

    Args:
        df (pandas.DataFrame): Dataframe with defined columns to adjust

    Returns:
        df (pandas.DataFrame): processed dataframe
    """
    columns_to_convert = ['fpedats','revdats', 'anndats', 'anndats_act']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column])
        
    return df

In [44]:
df = pd.read_parquet('../data/raw/ibes-forecasts.parquet')
df = df.rename(columns={col: col.lower() for col in df.columns})

df = convert_to_datetime(df)

df = df.rename(columns={"ticker": "ibes_ticker_pk", "oftic": "official_ticker", "analys": "analyst", 
                            "value": "estimated_eps", "fpedats": "fiscal_period_ending", 
                            "revdats": "revision_date", "anndats": "announce_date", 
                            "actual": "actual_eps", "anndats_act": "announce_date_actual", "cname":"company_name"
                            })
df['analyst_year'] = df.announce_date.dt.year
df_analyst = df[['analyst', 'announce_date']]
df_analyst['year'] = df_analyst['announce_date'].dt.year
df_analyst = df_analyst.drop(columns= ['announce_date'])
df_analyst = df_analyst.drop_duplicates().reset_index(drop=True)
df_analyst

/var/folders/q6/rg_7lxjs1wx6080czdx8tpjm0000gn/T/ipykernel_82546/2965240066.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analyst['year'] = df_analyst['announce_date'].dt.year


,analyst,year
0,119962,2014.0
1,80474,2014.0
2,50789,2014.0
3,79092,2014.0
4,71182,2014.0
...,...,...
68095,104805,2011.0
68096,104805,2012.0
68097,159645,2013.0
68098,131108,2019.0


### Step 2 - add historical data for analysts

In [45]:
old_data = pd.read_parquet('../data/raw/1990-2014.parquet')
old_data.head()

,TICKER,OFTIC,CNAME,ACTDATS,ANALYS,FPI,MEASURE,FPEDATS,ANNDATS
0,0000,TLMR,TALMER BANCORP,2014-03-11,119962,1,EPS,2014-12-31,2014-03-09
1,0000,TLMR,TALMER BANCORP,2014-03-11,80474,1,EPS,2014-12-31,2014-03-10
2,0000,TLMR,TALMER BANCORP,2014-03-11,50789,1,EPS,2014-12-31,2014-03-10
3,0000,TLMR,TALMER BANCORP,2014-03-11,79092,1,EPS,2014-12-31,2014-03-10
4,0000,TLMR,TALMER BANCORP,2014-03-17,71182,1,EPS,2014-12-31,2014-03-17


In [46]:
analysts_experience = old_data[['ANALYS', 'ANNDATS']].drop_duplicates().reset_index(drop=True)
analysts_experience

,ANALYS,ANNDATS
0,119962,2014-03-09
1,80474,2014-03-10
2,50789,2014-03-10
3,79092,2014-03-10
4,71182,2014-03-17
...,...,...
2138012,136140,2013-11-10
2138013,41097,2013-11-14
2138014,136140,2014-09-28
2138015,41097,2014-10-13


In [47]:
analysts_experience['ANNDATS'] = pd.to_datetime(analysts_experience['ANNDATS'])
analysts_experience['year'] = analysts_experience['ANNDATS'].dt.year
analysts_experience.head()

,ANALYS,ANNDATS,year
0,119962,2014-03-09,2014
1,80474,2014-03-10,2014
2,50789,2014-03-10,2014
3,79092,2014-03-10,2014
4,71182,2014-03-17,2014


In [48]:
analysts_experience.rename(columns={'ANALYS':'analyst'}, inplace=True)
analysts_experience.drop(columns='ANNDATS', inplace=True)
analysts_experience.head()

,analyst,year
0,119962,2014
1,80474,2014
2,50789,2014
3,79092,2014
4,71182,2014


### Step 4 - Concat old data with new data

In [49]:
all_years_analyst = pd.concat([analysts_experience, df_analyst])
all_years_analyst.drop_duplicates().reset_index(drop=True)
all_years_analyst.head()

,analyst,year
0,119962,2014.0
1,80474,2014.0
2,50789,2014.0
3,79092,2014.0
4,71182,2014.0


In [53]:
# years of experience is calculated as a current year - first mention of analyst in dataset
all_years_analyst['experience'] = all_years_analyst.groupby('analyst')['year'].transform(lambda x: x - x.min())

In [51]:
all_years_analyst = all_years_analyst.drop_duplicates()

In [52]:
all_years_analyst.to_parquet('../data/processed/analyst_experience.parquet')